In [1]:
import tensorflow as tf
from tensorflow.python.layers.core import Dense
import numpy as np
import os


print('TensorFlow Version: {}'.format(tf.__version__))

TensorFlow Version: 1.0.0


In [2]:
solution_netowrk = tf.Graph()
with solution_netowrk.as_default():
    # k,q,n_max,n
    # q,a,v
    regularizer1 = tf.contrib.layers.l2_regularizer(scale=0.1)
    regularizer2 = tf.contrib.layers.l2_regularizer(scale=0.1)
    board_00 = tf.placeholder(tf.float32, [1,3+10+10+11+11], name='board_00')
    target_00 = tf.placeholder(tf.float32, [1,11+11+2+2+1], name='target_00')
    nn001 = tf.layers.dense(board_00, 1000, activation=tf.nn.sigmoid, kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.5)
                           ,bias_initializer=tf.truncated_normal_initializer(mean = 0.0, stddev=0.5)
                           ,kernel_regularizer=regularizer1
                           ,bias_regularizer=regularizer2)
#     nn0012 = tf.layers.dense(nn001, 100, activation=tf.nn.sigmoid,kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))
#     nn0013 = tf.layers.dense(nn0012, 100, activation=tf.nn.sigmoid,kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))
#     nn0014 = tf.layers.dense(nn0013, 100, activation=tf.nn.sigmoid, kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))
    nn0015 = tf.layers.dense(nn001, 11+11+2+2+1, activation=tf.nn.sigmoid, kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.5))
    
    prior = solution_prior = tf.nn.softmax(nn0015[:,:26])
    v = solution_v = nn0015[:,26]

    pi = target_00[:,:26]
    z = target_00[:,26]
    
    reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
    reg_constant = 1  # Choose an appropriate one.
    l2_loss = reg_constant * sum(reg_losses)
    
    solution_cost = tf.reduce_mean((z - v)**2 + tf.nn.softmax_cross_entropy_with_logits(logits=prior, labels=pi) + l2_loss)
    solution_optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(solution_cost)
    
    init_00 = tf.global_variables_initializer()
    
solution_sess = tf.Session(graph=solution_netowrk)
solution_sess.run(init_00)

In [3]:
class HSRState:
    """ A state of the game, i.e. the game board.
        (k,q,n)
        k : number of jars left
        q : number of questions left
        n : number of total rungs 
    """
    def __init__(self):
        self.playerJustMoved = 2 # At the root pretend the player just moved is p2 - p1 has the first move
        self.board = [2,3,-1,-1] # k, q, n_max, n chosen as question
        self.reversedRole = False
        self.P2firstMove = True
        self.flag = -1
        
    def Clone(self):
        """ Create a deep clone of this game state.
        """
        st = HSRState()
        st.playerJustMoved = self.playerJustMoved
        st.board = self.board[:]
        st.reversedRole = self.reversedRole
        st.P2firstMove = self.P2firstMove
        st.flag = self.flag
        return st

    def DoMove(self, move):
        """ Update a state by carrying out the given move.
            Must update playerToMove.
        """
        k = self.board[0]
        q = self.board[1]
        n_max = self.board[2]
        n = self.board[3]
        assert n <= n_max
        if not self.reversedRole:
            if self.playerJustMoved == 2:
                if self.board[2] == -1:
                    self.board[2] = move
                else:
                    self.board[3] = move
            if self.playerJustMoved == 1:
                self.P2firstMove = False
                if move == 'reject':
                    self.board[2] = self.board[2] + 1
                    self.reversedRole = True
                    self.flag = 'reject'
                elif move == 'accept':
                    self.flag = 'accept'
                elif move == 'yes':
                    # break
                    k -= 1
                    q -= 1
                    n_max = n-1
                    n = -1
                    self.board = [k,q,n_max,n]
                elif move == 'no':
                    #not break
                    q -= 1
                    n_max = n_max-n
                    n = -1
                    self.board = [k,q,n_max,n]
        else:
            if self.playerJustMoved == 2:
                if move == 'yes':
                    # break
                    k -= 1
                    q -= 1
                    n_max = n-1
                    n = -1
                    self.board = [k,q,n_max,n]

                elif move == 'no':
                    #not break
                    q -= 1
                    n_max = n_max-n
                    n = -1
                    self.board = [k,q,n_max,n]
                elif move == 'pass':
                    self.flag = 'pass'
            if self.playerJustMoved == 1:
                self.P2firstMove = False
                self.board[3] = move
                self.flag = 'reject'

        self.playerJustMoved = 3 - self.playerJustMoved
        
    def GetMoves(self):
        """ Get all possible moves from this state.
        """
        k = self.board[0]
        q = self.board[1]
        n_max = self.board[2]
        n = self.board[3]
        if not self.reversedRole:
            if n_max == 0:
                return []
            if k==0 or q==0:
                return []
            if self.playerJustMoved == 2:
                if n_max == -1:
                    return range(1,12)
                return range(1,n_max+1)
            if self.playerJustMoved == 1:
                if self.P2firstMove:
                    if n_max<11:
                        return ['accept','reject']
                    else:
                        return ['accept']
                return ['yes','no']
        else:
            if n_max == 0:
                return []
            if k==0 or q==0:
                return []
            if self.playerJustMoved == 1:
                return range(1,n_max+1)
            if self.playerJustMoved == 2:
                if n == -1:
                    return ['pass']
                return ['yes','no']

    
    def GetResult(self, playerjm):
        """ Get the game result from the viewpoint of playerjm. 
        """
        self.P2firstMove = True
        n_max = self.board[2]
        if not self.reversedRole:
            if playerjm == 1:
                return 1 if n_max == 0 else 0
            if playerjm == 2:
                return 0 if n_max == 0 else 1
        else:
            if playerjm == 1:
                return 0 if n_max == 0 else 1
            if playerjm == 2:
                return 1 if n_max == 0 else 0


    def __repr__(self):
        s = ''
        for x in self.board:
            s += str(x) +','

        return s[:-1]

In [4]:
def neural_evaluator(state):
    k,q,n_max,n = state.board
    flag = state.flag
    prior, v = solution_sess.run([solution_prior,solution_v],{board_00: [getBoardOneHot2(flag,k,q,n_max,n)]})
    return prior , v

In [5]:
# def getBoardOneHot0(k,q):
#     return getBoardOneHot2(k,q,-1,-1)
# def getBoardOneHot(k,q,n_max):
#     return getBoardOneHot2(k,q,n_max,-1)
def getBoardOneHot2(flag,k,q,n_max,n):
    # [None,10+10+100+100]
    f0 = np.zeros(shape=(3), dtype=np.float32)
    k0 = np.zeros(shape=(10), dtype=np.float32)
    q0 = np.zeros(shape=(10), dtype=np.float32)
    n_max0 = np.zeros(shape=(11), dtype=np.float32)
    n0 = np.zeros(shape=(11), dtype=np.float32)
    if flag !=-1:
        if flag == 'reject':
            f0[1-1] = 0
        elif flag == 'accept':
            f0[2-1] = 0
        elif flag == 'pass':
            f0[3-1] = 0
    if k != -1:
        k0[k-1] = 1
    if q != -1:
        q0[q-1] = 1
    if n_max != -1:
        n_max0[n_max-1]=1
    if n!= -1:
        n0[n-1] = 1
    return np.concatenate((f0,k0, q0, n_max0, n0), axis=0)
    

In [6]:
import random
def moveSampler(notes, pi):
    x = random.random()
    acc = 0
    for index in range(len(pi)):
        if x <= acc + pi[index]:
            return notes[index].move
        acc += pi[index]

In [8]:
from math import *
import random

class GameState:
    """ A state of the game, i.e. the game board. These are the only functions which are
        absolutely necessary to implement PUCT in any 2-player complete information deterministic 
        zero-sum game, although they can be enhanced and made quicker, for example by using a 
        GetRandomMove() function to generate a random move during rollout.
        By convention the players are numbered 1 and 2.
    """
    def __init__(self):
            self.playerJustMoved = 2 # At the root pretend the player just moved is player 2 - player 1 has the first move
        
    def Clone(self):
        """ Create a deep clone of this game state.
        """
        st = GameState()
        st.playerJustMoved = self.playerJustMoved
        return st

    def DoMove(self, move):
        """ Update a state by carrying out the given move.
            Must update playerJustMoved.
        """
        self.playerJustMoved = 3 - self.playerJustMoved
        
    def GetMoves(self):
        """ Get all possible moves from this state.
        """
    
    def GetResult(self, playerjm):
        """ Get the game result from the viewpoint of playerjm. 
        """

    def __repr__(self):
        """ Don't need this - but good style.
        """
        pass

class Node:
    """ A node in the game tree. Note wins is always from the viewpoint of playerJustMoved.
        Crashes if state not specified.
    """
    def __init__(self, move = None, parent = None, state = None):
        self.move = move # the move that got us to this node - "None" for the root node
        self.parentNode = parent # "None" for the root node
        self.childNodes = []
        self.prior = []
        self.wins = 0
        self.visits = 0
        self.untriedMoves = state.GetMoves() # future child nodes
        self.playerJustMoved = state.playerJustMoved # the only part of the state that the Node needs later
    
    def PUCTSelectChild_vanilla(self):
        """ Use the PUCK formula to select a child node. Often a constant PUCTK is applied so we have
            lambda c: Q + U to vary the amount of exploration versus exploitation.

            Q = c.wins/c.visits
            U = PUCTK * prior * sqrt(self.visits)/c.visits

            (according to the AplhaZero paper)

        """
        PUCTK = 1
        prior = 1.0/len(self.childNodes) # TODO this is a constant now, need retrieve from neural network.
#         s = sorted(self.childNodes, key = lambda c: c.wins*1.0/c.visits+ PUCTK * prior * sqrt(self.visits)/(1+c.visits))[-1]
        s = sorted(self.childNodes, key = lambda c: c.wins*1.0/c.visits + sqrt(2*log(self.visits)/c.visits))[-1]
        return s
        
    def PUCTSelectChild(self):
        """ Use the PUCK formula to select a child node. Often a exploration factor PUCTK is applied so we have
            lambda c: Q + U to vary the amount of exploration versus exploitation.

            Q = c.wins/c.visits
            U = PUCTK * prior * sqrt(self.visits)/c.visits

            (according to the AplhaZero paper)

        """
        PUCTK = 1
#         prior = [1.0/len(self.childNodes)]*(len(self.childNodes))
        x = zip(sorted(self.childNodes, key = lambda c:c.move), self.prior[:len(self.childNodes)])
#         x = zip(sorted(self.childNodes, key = lambda c:c.move), prior)
        s = sorted(x, key = lambda c: c[0].wins*1.0/c[0].visits+ PUCTK * c[1] * sqrt(self.visits)/(1+c[0].visits))[-1]# if c[0].visits!=0 else PUCTK * c[1] * sqrt(self.visits)/(1+c[0].visits))[-1]
#         print s
        return s[0]
    
    def AddChild(self, m, s):
        """ Remove m from untriedMoves and add a new child node for this move.
            Return the added child node
        """
        n = Node(move = m, parent = self, state = s)
        self.untriedMoves.remove(m)
        self.childNodes.append(n)
        return n
    
    def Update(self, result):
        """ Update this node - one additional visit and result additional wins. result must be from the viewpoint of playerJustmoved.
        """
        self.visits += 1
        self.wins += 1 if result>=0.5 else 0
#         self.wins += result

    def __repr__(self):
        return "[M:" + str(self.move) + " W/V:" + str(self.wins) + "/" + str(self.visits) + " U:" + str(self.untriedMoves) + "]"

    def TreeToString(self, indent):
        s = self.IndentString(indent) + str(self)
        for c in self.childNodes:
             s += c.TreeToString(indent+1)
        return s

    def IndentString(self,indent):
        s = "\n"
        for i in range (1,indent+1):
            s += "| "
        return s

    def ChildrenToString(self):
        s = ""
        for c in self.childNodes:
             s += str(c) + "\n"
        return s
    
    def PriorToString(self):
        s = ""
        for c in self.prior:
             s += str(c) + "\n"
        return s
    
def PUCT_vanilla(rootstate, itermax, verbose = False):
    """ Conduct a PUCT search for itermax iterations starting from rootstate.
        Return the best move from the rootstate.
        Assumes 2 alternating players (player 1 starts), with game results in the range [0.0, 1.0]."""

    rootnode = Node(state = rootstate)

    for i in range(itermax):
        node = rootnode
        state = rootstate.Clone()

        # Select
        while node.untriedMoves == [] and node.childNodes != []: # node is fully expanded and non-terminal
            node = node.PUCTSelectChild_vanilla()
            state.DoMove(node.move)

        # Expand
        if node.untriedMoves != []: # if we can expand (i.e. state/node is non-terminal)
            m = random.choice(node.untriedMoves) 
            state.DoMove(m)
            node = node.AddChild(m,state) # add child and descend tree

        # Rollout - this can often be made orders of magnitude quicker using a state.GetRandomMove() function
        while state.GetMoves() != []: # while state is non-terminal
            state.DoMove(random.choice(state.GetMoves()))

        # Backpropagate
        while node != None: # backpropagate from the expanded node and work back to the root node
            node.Update(state.GetResult(node.playerJustMoved)) # state is terminal. Update node with result from POV of node.playerJustMoved
            node = node.parentNode

    # Output some information about the tree - can be omitted
    # if (verbose): print rootnode.TreeToString(0)
    # else: print rootnode.ChildrenToString()
    
    pi = map(lambda c:(c.visits)*1.0/(rootnode.visits), sorted(rootnode.childNodes, key = lambda c:c.move))

    return sorted(rootnode.childNodes, key = lambda c: c.visits)[-1].move, pi # return the move that was most visited


def PUCT(rootstate, itermax, verbose = False):
    """ Conduct a PUCT search for itermax iterations starting from rootstate.
        Return the best move from the rootstate.
        Assumes 2 alternating players (player 1 starts), with game results in the range [0.0, 1.0]."""

    rootnode = Node(state = rootstate)

    for i in range(itermax):
#         print('iter:{}'.format(i))
        node = rootnode
        state = rootstate.Clone()
        prior, v = neural_evaluator(state)
        if not state.reversedRole:
            if state.playerJustMoved == 2:
                # player 1 play
                k,q,n_max,n = state.board
                if n_max == -1 and n == -1:
                    # get prior for n_max
                    node.prior = prior[0][:11]
                else:
                    # get prior for n
                    node.prior = prior[0][11:11+11]
            else:
                # player 2 play
                k,q,n_max,n = state.board
                if n == -1:
                    # get prior for accept/reject
                    node.prior = prior[0][11+13:11+15]
                else:
                    # get prior for yes/no
                    node.prior = prior[0][11+11:11+13]
        else:
            if state.playerJustMoved == 2:
                # player 1 play
                k,q,n_max,n = state.board
                if n == -1:
                    node.prior = [1.0]
                else:
                    # get prior for yes/no
                    node.prior = prior[0][11+11:11+13]
            else:
                # player 2 play
                # get prior for n
                node.prior = prior[0][11:11+11]

        # Select
        while node.untriedMoves == [] and node.childNodes != []: # node is fully expanded and non-terminal
#             print str(state.board)+":"+str(state.flag)
            node = node.PUCTSelectChild()
#             print
            state.DoMove(node.move)

        # Expand and Evaluate
                
        if node.untriedMoves != []: # if we can expand (i.e. state/node is non-terminal)
            m = random.choice(node.untriedMoves) 
            state.DoMove(m)
            node = node.AddChild(m,state) # add child and descend tree

        prior, v = neural_evaluator(state)
        if not state.reversedRole:
            if state.playerJustMoved == 2:
                # player 1 play
                k,q,n_max,n = state.board
                if n_max == -1 and n == -1:
                    # get prior for n_max
                    node.prior = prior[0][:11]
                else:
                    # get prior for n
                    node.prior = prior[0][11:11+11]
            else:
                # player 2 play
                k,q,n_max,n = state.board
                if n == -1:
                    # get prior for accept/reject
                    node.prior = prior[0][11+13:11+15]
                else:
                    # get prior for yes/no
                    node.prior = prior[0][11+11:11+13]
        else:
            if state.playerJustMoved == 2:
                # player 1 play
                k,q,n_max,n = state.board
                if n == -1:
                    node.prior = [1.0]
                else:
                    # get prior for yes/no
                    node.prior = prior[0][11+11:11+13]
            else:
                # player 2 play
                # get prior for n
                node.prior = prior[0][11:11+11]
                
#         while node.untriedMoves != []: # if we can expand (i.e. state/node is non-terminal)
#             m = random.choice(node.untriedMoves)
#             state_next = state.Clone()
#             state_next.DoMove(m)
#             c_node = node.AddChild(m,state_next) # add child and descend tree
#             # Backpropagate
#             while c_node != None: # backpropagate from the expanded node and work back to the root node
#                 c_node.Update(v) # state is terminal. Update node with result from POV of node.playerJustMoved
#                 c_node = c_node.parentNode
            
        # Backpropagate
        while node != None: # backpropagate from the expanded node and work back to the root node
            node.Update(v) # state is terminal. Update node with result from POV of node.playerJustMoved
            node = node.parentNode
        
    # Output some information about the tree - can be omitted
    # if (verbose): print rootnode.TreeToString(0)
    # else: print rootnode.ChildrenToString()
    
#     print rootnode.TreeToString(0)
    
    # Here we should return distribution of the moves:
    # TODO: need use sampling instead of maximum.
    root_visit = sum(map(lambda c:c.visits,rootnode.childNodes))
    tau = 1
    pi = map(lambda c:(c.visits**tau)*1.0/(rootnode.visits**tau), sorted(rootnode.childNodes, key = lambda c:c.move))
    
    move = moveSampler(sorted(rootnode.childNodes, key = lambda c:c.move),pi)
    
#     print pi
    
#     print sorted(rootnode.childNodes, key = lambda c: c.visits)

    xx = random.random()
    if xx <= 0.99999:
        return sorted(rootnode.childNodes, key = lambda c: c.visits)[-1].move, pi
    else:
        return move,pi

def PUCTPlayGame():
    """ Play a sample game between two PUCT players where each player gets a different number 
        of PUCT iterations (= simulations = tree nodes).
    """
    winner = -1
    state = HSRState()
    
    history_s = []
    history_pi=[]
    history_z=[]
    
    while (state.GetMoves() != []):
#         print(str(state))
        if state.playerJustMoved == 1:
            m,pi = PUCT(rootstate = state, itermax = 1000, verbose = False) # play with values for itermax and verbose = True
        else:
            m,pi = PUCT(rootstate = state, itermax = 1000, verbose = False)
        print(pi)
        print("Player "+str(3-state.playerJustMoved)+" Move: " + str(m))
        history_s.append(state.Clone())
        history_pi.append(pi)
        state.DoMove(m)
    if state.GetResult(state.playerJustMoved) == 1.0:
#         print("Player " + str(state.playerJustMoved) + " wins!")
        winner = state.playerJustMoved
    elif state.GetResult(state.playerJustMoved) == 0.0:
#         print("Player " + str(3 - state.playerJustMoved) + " wins!")
        winner = 3 - state.playerJustMoved
    else: print("Nobody wins!")

    assert len(history_s) == len(history_pi)
    
    for step in reversed(range(len(history_s))):
        if winner == state.playerJustMoved:
            if (len(history_s) - step) % 2 == 1:
                history_z.append(1)
            else:
                history_z.append(0)
        else:
            if (len(history_s) - step) % 2 == 1:
                history_z.append(0)
            else:
                history_z.append(1)
    assert len(history_s) == len(history_z)
    history_z.reverse() 
    return winner , history_s, history_pi ,history_z

# we can train the neural network only after getting the result
# according to the AlphaZero paper, we need (s,pi,z)

result = {1:0,2:0}
board_record = {}
for simulation in range(100000):
    winner, history_s, history_pi ,history_z = PUCTPlayGame()
    result[winner] += 1
    print str(result)+"\n"
#     print board_record
#     print history_z
    for t in range(len(history_s)):
        state = history_s[t]
        pi = history_pi[t]
        z = history_z[t]

        if not state.reversedRole:
            if state.playerJustMoved == 2:
                # player 1 play
                k,q,n_max,n = state.board
                f = state.flag
                if n_max==-1 and n==-1:
                    # set prior for n_max
                    pi = pi+[0]*11+[0]*(15-len(history_pi[t]))
                else:
                    # set prior for n 
                    pi = [0]*11+pi+[0]*(15-len(history_pi[t]))
#                 print pi
                pp,v = solution_sess.run([solution_prior,solution_v],{board_00: [getBoardOneHot2(f,k,q,n_max,n)]})
#                 print str(pp[0])
                print str(z)+":"+str(v)+":"+str(state.board)+":"+str(f)
                _, costt = solution_sess.run([solution_optimizer,solution_cost],{board_00: [getBoardOneHot2(f,k,q,n_max,n)], target_00: [pi+[z]]})
                print("question cost:"+str(costt))
            else:
                # player 2 play
                k,q,n_max,n = state.board
                f = state.flag
                if n==-1:
                    # set prior for accept/reject
                    pi = [0]*11+[0]*(13)+pi+[0]*(2-len(history_pi[t]))
                else:
                    # set prior for yes/no
                    pi = [0]*11+[0]*(11)+pi+[0]*(2-len(history_pi[t]))+[0]*(2)
#                 print pi
                pp,v = solution_sess.run([solution_prior,solution_v],{board_00: [getBoardOneHot2(f,k,q,n_max,n)]})
#                 print str(pp[0])
                print str(z)+":"+str(v)+":"+str(state.board)+":"+str(f)
                _, costt = solution_sess.run([solution_optimizer,solution_cost],{board_00: [getBoardOneHot2(f,k,q,n_max,n)], target_00: [pi+[z]]})
                print("answer cost:"+str(costt))
        else:
            if state.playerJustMoved == 2:
                # player 1 play
                k,q,n_max,n = state.board
                f = state.flag
                if n==-1:
                    continue
                pi = [0]*11+[0]*(11)+pi+[0]*(2-len(history_pi[t]))+[0]*(2)
#                 print pi
                pp,v = solution_sess.run([solution_prior,solution_v],{board_00: [getBoardOneHot2(f,k,q,n_max,n)]})
#                 print str(pp[0])
                print str(z)+":"+str(v)+":"+str(state.board)+":"+str(f)
                _, costt = solution_sess.run([solution_optimizer,solution_cost],{board_00: [getBoardOneHot2(f,k,q,n_max,n)], target_00: [pi+[z]]})
                print("force answer cost:"+str(costt))
            else:
                # player 2 play
                # player 1 play
                k,q,n_max,n = state.board
                f = state.flag
                pi = [0]*11+pi+[0]*(15-len(history_pi[t]))
#                 print pi
                pp,v = solution_sess.run([solution_prior,solution_v],{board_00: [getBoardOneHot2(f,k,q,n_max,n)]})
#                 print str(pp[0])
                print str(z)+":"+str(v)+":"+str(state.board)+":"+str(f)
                _, costt = solution_sess.run([solution_optimizer,solution_cost],{board_00: [getBoardOneHot2(f,k,q,n_max,n)], target_00: [pi+[z]]})
                print("question cost:"+str(costt))
                
# question_sess.close()
# answer_sess.close()

# print result
            
    

[0.019, 0.051, 0.082, 0.113, 0.008, 0.206, 0.103, 0.011, 0.103, 0.144, 0.16]
Player 1 Move: 6
[0.398, 0.602]
Player 2 Move: reject
[1.0]
Player 1 Move: pass
[0.203, 0.194, 0.177, 0.1, 0.083, 0.126, 0.117]
Player 2 Move: 1
[0.877, 0.123]
Player 1 Move: no
[0.112, 0.208, 0.247, 0.151, 0.112, 0.17]
Player 2 Move: 3
[0.46, 0.54]
Player 1 Move: yes
[0.205, 0.795]
Player 2 Move: 2
[0.269, 0.731]
Player 1 Move: yes
{1: 1, 2: 0}

1:[ 0.41816771]:[2, 3, -1, -1]:-1
question cost:3.64767
0:[ 0.37743768]:[2, 3, 6, -1]:-1
answer cost:3.40427
0:[ 0.30621499]:[2, 3, 7, -1]:pass
question cost:3.37931
1:[ 0.29405051]:[2, 3, 7, 1]:reject
force answer cost:3.80153
0:[ 0.27033615]:[2, 2, 6, -1]:reject
question cost:3.35214
1:[ 0.68913227]:[2, 2, 6, 3]:reject
force answer cost:3.37942
0:[ 0.2801187]:[1, 1, 2, -1]:reject
question cost:3.34912
1:[ 0.71791774]:[1, 1, 2, 2]:reject
force answer cost:3.35318
[0.014, 0.023, 0.075, 0.068, 0.134, 0.16, 0.174, 0.067, 0.006, 0.11, 0.169]
Player 1 Move: 7
[0.403, 0.59

[1.0]
Player 2 Move: accept
[0.108, 0.101, 0.138, 0.035, 0.047, 0.162, 0.111, 0.12, 0.059, 0.084, 0.035]
Player 1 Move: 6
[0.579, 0.421]
Player 2 Move: no
[0.148, 0.277, 0.277, 0.149, 0.149]
Player 1 Move: 2
[0.549, 0.451]
Player 2 Move: no
[0.114, 0.443, 0.443]
Player 1 Move: 3
[0.269, 0.731]
Player 2 Move: yes
{1: 8, 2: 2}

0:[ 0.4352949]:[2, 3, -1, -1]:-1
question cost:3.50132
1:[ 0.43629614]:[2, 3, 11, -1]:-1
answer cost:3.58228
0:[ 0.44859585]:[2, 3, 11, -1]:accept
question cost:3.49407
1:[ 0.57502413]:[2, 3, 11, 6]:accept
answer cost:3.47228
0:[ 0.2829012]:[2, 2, 5, -1]:accept
question cost:3.36538
1:[ 0.73887545]:[2, 2, 5, 2]:accept
answer cost:3.3583
0:[ 0.27532738]:[2, 1, 3, -1]:accept
question cost:3.34535
1:[ 0.69336498]:[2, 1, 3, 3]:accept
answer cost:3.37086
[0.018, 0.049, 0.037, 0.106, 0.175, 0.211, 0.008, 0.097, 0.096, 0.008, 0.195]
Player 1 Move: 6
[0.424, 0.576]
Player 2 Move: reject
[1.0]
Player 1 Move: pass
[0.12, 0.242, 0.187, 0.106, 0.088, 0.132, 0.125]
Player 2 Mo

[0.889, 0.111]
Player 1 Move: no
[0.077, 0.17, 0.169, 0.13, 0.13, 0.17, 0.077, 0.077]
Player 2 Move: 2
[0.621, 0.379]
Player 1 Move: no
[0.068, 0.265, 0.265, 0.068, 0.068, 0.266]
Player 2 Move: 6
[0.269, 0.731]
Player 1 Move: yes
{1: 15, 2: 4}

1:[ 0.48793933]:[2, 3, -1, -1]:-1
question cost:3.57228
0:[ 0.35549876]:[2, 3, 10, -1]:-1
answer cost:3.3893
0:[ 0.58326256]:[2, 3, 11, -1]:pass
question cost:3.62922
1:[ 0.86466795]:[2, 3, 11, 3]:reject
force answer cost:3.32342
0:[ 0.30357724]:[2, 2, 8, -1]:reject
question cost:3.37836
1:[ 0.7612254]:[2, 2, 8, 2]:reject
force answer cost:3.34927
0:[ 0.27999562]:[2, 1, 6, -1]:reject
question cost:3.35091
1:[ 0.45142749]:[2, 1, 6, 6]:reject
force answer cost:3.57644
[0.014, 0.014, 0.06, 0.006, 0.199, 0.087, 0.006, 0.187, 0.109, 0.098, 0.22]
Player 1 Move: 11
[1.0]
Player 2 Move: accept
[0.009, 0.167, 0.123, 0.031, 0.162, 0.145, 0.118, 0.086, 0.086, 0.053, 0.02]
Player 1 Move: 2
[0.927, 0.073]
Player 2 Move: no
[0.022, 0.136, 0.162, 0.124, 0.124,

[0.448, 0.552]
Player 2 Move: reject
[1.0]
Player 1 Move: pass
[0.105, 0.204, 0.188, 0.105, 0.105, 0.143, 0.12, 0.03]
Player 2 Move: 2
[0.884, 0.116]
Player 1 Move: no
[0.107, 0.201, 0.238, 0.145, 0.108, 0.201]
Player 2 Move: 3
[0.46, 0.54]
Player 1 Move: yes
[0.205, 0.795]
Player 2 Move: 2
[0.269, 0.731]
Player 1 Move: yes
{1: 23, 2: 5}

1:[ 0.4516747]:[2, 3, -1, -1]:-1
question cost:3.61104
0:[ 0.32829612]:[2, 3, 7, -1]:-1
answer cost:3.37097
0:[ 0.34383336]:[2, 3, 8, -1]:pass
question cost:3.40357
1:[ 0.80279845]:[2, 3, 8, 2]:reject
force answer cost:3.3442
0:[ 0.34314859]:[2, 2, 6, -1]:reject
question cost:3.39841
1:[ 0.67979336]:[2, 2, 6, 3]:reject
force answer cost:3.38799
0:[ 0.26434711]:[1, 1, 2, -1]:reject
question cost:3.34348
1:[ 0.73688233]:[1, 1, 2, 2]:reject
force answer cost:3.346
[0.014, 0.045, 0.045, 0.113, 0.106, 0.052, 0.193, 0.054, 0.144, 0.006, 0.228]
Player 1 Move: 11
[1.0]
Player 2 Move: accept
[0.01, 0.161, 0.148, 0.035, 0.143, 0.161, 0.143, 0.083, 0.01, 0.071, 

[1.0]
Player 1 Move: pass
[0.12, 0.242, 0.187, 0.106, 0.088, 0.132, 0.125]
Player 2 Move: 2
[0.845, 0.155]
Player 1 Move: no
[0.148, 0.277, 0.277, 0.149, 0.149]
Player 2 Move: 2
[0.549, 0.451]
Player 1 Move: no
[0.114, 0.443, 0.443]
Player 2 Move: 2
[0.269, 0.731]
Player 1 Move: yes
{1: 29, 2: 8}

1:[ 0.37672287]:[2, 3, -1, -1]:-1
question cost:3.6977
0:[ 0.38371336]:[2, 3, 6, -1]:-1
answer cost:3.40922
0:[ 0.25390357]:[2, 3, 7, -1]:pass
question cost:3.34722
1:[ 0.72825187]:[2, 3, 7, 2]:reject
force answer cost:3.37582
0:[ 0.32749575]:[2, 2, 5, -1]:reject
question cost:3.39041
1:[ 0.78647685]:[2, 2, 5, 2]:reject
force answer cost:3.33361
0:[ 0.39677858]:[2, 1, 3, -1]:reject
question cost:3.42493
1:[ 0.82552981]:[2, 1, 3, 2]:reject
force answer cost:3.30521
[0.016, 0.006, 0.062, 0.006, 0.161, 0.006, 0.114, 0.258, 0.178, 0.006, 0.187]
Player 1 Move: 8
[0.408, 0.592]
Player 2 Move: reject
[1.0]
Player 1 Move: pass
[0.012, 0.186, 0.23, 0.04, 0.054, 0.159, 0.111, 0.097, 0.111]
Player 2 Mov

...

[0.009, 0.009, 0.009, 0.035, 0.122, 0.134, 0.009, 0.084, 0.146, 0.183, 0.26]
Player 1 Move: 11
[1.0]
Player 2 Move: accept
[0.109, 0.249, 0.249, 0.059, 0.084, 0.064, 0.047, 0.035, 0.01, 0.035, 0.059]
Player 1 Move: 2
[0.972, 0.028]
Player 2 Move: no
[0.705, 0.04, 0.077, 0.005, 0.018, 0.04, 0.067, 0.043, 0.005]
Player 1 Move: 1
[0.996, 0.004]
Player 2 Move: no
[0.001, 0.966, 0.003, 0.001, 0.009, 0.01, 0.001, 0.009]
Player 1 Move: 2
[0.998, 0.002]
Player 2 Move: no
{1: 1266, 2: 998}

0:[ 0.39885181]:[2, 3, -1, -1]:-1
question cost:3.44296
1:[ 0.4771539]:[2, 3, 11, -1]:-1
answer cost:3.51227
0:[ 0.52050239]:[2, 3, 11, -1]:accept
question cost:3.52994
1:[ 0.55280459]:[2, 3, 11, 2]:accept
answer cost:3.43941
0:[ 0.45240918]:[2, 2, 9, -1]:accept
question cost:3.48243
1:[ 0.41974428]:[2, 2, 9, 1]:accept
answer cost:3.57655
0:[ 0.52394265]:[2, 1, 8, -1]:accept
question cost:3.51554
1:[ 0.55955869]:[2, 1, 8, 2]:accept
answer cost:3.43423
[0.001, 0.014, 0.013, 0.013, 0.785, 0.019, 0.022, 0.023, 

[0.002, 0.998]
Player 2 Move: yes
[0.001, 0.003, 0.996]
Player 1 Move: 3
[0.002, 0.998]
Player 2 Move: yes
{1: 1275, 2: 1003}

0:[ 0.45067266]:[2, 3, -1, -1]:-1
question cost:3.49261
1:[ 0.46596238]:[2, 3, 5, -1]:-1
answer cost:3.53006
0:[ 0.46916163]:[2, 3, 5, -1]:accept
question cost:3.50848
1:[ 0.44117889]:[2, 3, 5, 4]:accept
answer cost:3.55792
0:[ 0.43060547]:[1, 2, 3, -1]:accept
question cost:3.43142
1:[ 0.3983641]:[1, 2, 3, 3]:accept
answer cost:3.60826
[0.046, 0.114, 0.005, 0.005, 0.8, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005]
Player 1 Move: 5
[0.121, 0.879]
Player 2 Move: reject
[1.0]
Player 1 Move: pass
[0.001, 0.002, 0.003, 0.001, 0.001, 0.992]
Player 2 Move: 6
[0.002, 0.998]
Player 1 Move: yes
[0.001, 0.003, 0.993, 0.002, 0.001]
Player 2 Move: 3
[0.004, 0.996]
Player 1 Move: yes
{1: 1276, 2: 1003}

1:[ 0.37728801]:[2, 3, -1, -1]:-1
question cost:3.67952
0:[ 0.50285619]:[2, 3, 5, -1]:-1
answer cost:3.49963
0:[ 0.38902262]:[2, 3, 6, -1]:pass
question cost:3.39821
1:[ 0.334773

[0.005, 0.995]
Player 1 Move: yes
{1: 1282, 2: 1009}

0:[ 0.55527496]:[2, 3, -1, -1]:-1
question cost:3.58536
1:[ 0.50984132]:[2, 3, 3, -1]:-1
answer cost:3.47222
1:[ 0.487515]:[2, 3, 4, -1]:pass
question cost:3.53974
0:[ 0.46716067]:[2, 3, 4, 1]:reject
force answer cost:3.45072
[0.001, 0.001, 0.015, 0.001, 0.028, 0.001, 0.001, 0.001, 0.949, 0.001, 0.001]
Player 1 Move: 9
[0.998, 0.002]
Player 2 Move: accept
[0.001, 0.002, 0.006, 0.028, 0.012, 0.944, 0.001, 0.001, 0.005]
Player 1 Move: 6
[0.006, 0.994]
Player 2 Move: yes
[0.001, 0.002, 0.003, 0.992, 0.002]
Player 1 Move: 4
[0.002, 0.998]
Player 2 Move: yes
{1: 1282, 2: 1010}

0:[ 0.47799197]:[2, 3, -1, -1]:-1
question cost:3.50649
1:[ 0.47580853]:[2, 3, 9, -1]:-1
answer cost:3.50797
0:[ 0.47730702]:[2, 3, 9, -1]:accept
question cost:3.46363
1:[ 0.33361548]:[2, 3, 9, 6]:accept
answer cost:3.67804
0:[ 0.47718593]:[1, 2, 5, -1]:accept
question cost:3.50667
1:[ 0.45782536]:[1, 2, 5, 4]:accept
answer cost:3.52791
[0.017, 0.038, 0.06, 0.06, 

[0.005, 0.005, 0.005, 0.143, 0.281, 0.005, 0.005, 0.005, 0.005, 0.005, 0.536]
Player 1 Move: 11
[1.0]
Player 2 Move: accept
[0.001, 0.004, 0.901, 0.023, 0.001, 0.021, 0.001, 0.001, 0.045, 0.001, 0.001]
Player 1 Move: 3
[0.997, 0.003]
Player 2 Move: no
[0.001, 0.002, 0.986, 0.006, 0.001, 0.002, 0.001, 0.001]
Player 1 Move: 3
[0.998, 0.002]
Player 2 Move: no
[0.001, 0.002, 0.003, 0.993, 0.001]
Player 1 Move: 4
[0.002, 0.998]
Player 2 Move: yes
{1: 1288, 2: 1016}

0:[ 0.42831761]:[2, 3, -1, -1]:-1
question cost:3.47634
1:[ 0.39185235]:[2, 3, 11, -1]:-1
answer cost:3.61721
0:[ 0.39546722]:[2, 3, 11, -1]:accept
question cost:3.40678
1:[ 0.60686678]:[2, 3, 11, 3]:accept
answer cost:3.40129
0:[ 0.38746056]:[2, 2, 8, -1]:accept
question cost:3.39719
1:[ 0.59927762]:[2, 2, 8, 3]:accept
answer cost:3.40703
0:[ 0.35091802]:[2, 1, 5, -1]:accept
question cost:3.41454
1:[ 0.32575971]:[2, 1, 5, 4]:accept
answer cost:3.70098
[0.015, 0.034, 0.053, 0.053, 0.034, 0.098, 0.352, 0.221, 0.034, 0.034, 0.072]

KeyboardInterrupt: 